In [ ]:
def estimate_vram_usage(
    num_params=27e9,             # Общее количество параметров модели (например, 27B для Gemma 3 27B)
    dtype_size=2,                # Размер хранения одного значения параметра (2 байта для fp16/bf16, 1 для 8bit)
    optimizer_multiplier=1.0,    # Множитель для состояния оптимизатора:
                                 #    - ~4.0 для Adam (fp32)
                                 #    - ~2.0 для 8-bit оптимизаторов (AdamW 8bit)
    grad_multiplier=0.5,         # Множитель для градиентов:
                                 #    - 1.0 для fp16/bf16
                                 #    - ~0.5 при использовании 8bit трюков
    batch_size=16,               # Размер батча на одно устройство (без учёта градиентного накопления)
    seq_len=8192,                # Длина входной последовательности (токенов)
    hidden_size=7168,            # Размер скрытого слоя (hidden size)
    num_layers=64,               # Количество слоёв трансформера
    activation_dtype_size=1,     # Размер хранения одного значения активации (2 байта для fp16/bf16)
    gradient_checkpointing=False  # Используется ли градиентный чекпоинтинг (снижает потребление активаций)
):
    # 💾 Память, занимаемая параметрами модели (только forward веса)
    model_memory = num_params * dtype_size

    # 💾 Память, занимаемая градиентами (для backward pass)
    grad_memory = num_params * dtype_size * grad_multiplier

    # 💾 Память, занимаемая состоянием оптимизатора (например, Adam требует хранить момент и variance)
    optimizer_memory = num_params * dtype_size * optimizer_multiplier

    # 💾 Память на активации (зависит от batch, seq_len, hidden_size, num_layers)
    act_memory = batch_size * seq_len * hidden_size * num_layers * activation_dtype_size

    # Если включён gradient checkpointing — пересчитываем активации, уменьшая потребление памяти (~65% экономия)
    if gradient_checkpointing:
        act_memory *= 0.35  # оставляем только 35% памяти на активации

    # 🧠 Общая память (в байтах)
    total_memory = model_memory + grad_memory + optimizer_memory + act_memory

    # 📊 Переводим всё в гигабайты (GB)
    return {
        "model_memory_GB": model_memory / 1e9,
        "grad_memory_GB": grad_memory / 1e9,
        "optimizer_memory_GB": optimizer_memory / 1e9,
        "activations_memory_GB": act_memory / 1e9,
        "total_GB": total_memory / 1e9
    }


In [4]:
vram = estimate_vram_usage(
    num_params=27e9,           # Gemma 3 27B
    dtype_size=1,              # 8-bit параметры (например, с BitsAndBytes)
    optimizer_multiplier=1.0,  # 8-bit оптимайзер (adamw_8bit)
    grad_multiplier=0.5,       # если градиенты хранятся в 8-bit или bf16 trick
    batch_size=16,
    seq_len=2048,
    hidden_size=7168,
    num_layers=64,
    activation_dtype_size=2,   # fp16/bf16
    gradient_checkpointing=True
)

import pprint
pprint.pprint(vram)


{'activations_memory_GB': 10.522669875199998,
 'grad_memory_GB': 13.5,
 'model_memory_GB': 27.0,
 'optimizer_memory_GB': 27.0,
 'total_GB': 78.0226698752}
